# The Artist Class Explained

The goal of the artist is to identify which buoy photo sets are the most beautiful or interesting.
The artist will be able to select a photo set and then rate it on a scale of 1-5.

the photo sets are located in the images/panels directory and are named by the buoy id
over time the artist will learn which times of day are the most beautiful and interesting for each buoy and will be able to teach the system to automatically raise the rate of photo capture during those times. For example, the most beautiful photos could be those that are taken at sunrise and sunset. The artist could also teach the system to take more photos when the weather is clear and less photos when the weather is cloudy. Unless cloudy weather is the most beautiful weather. That is up for debate.

As the system collects data about the buoy's photos it will begin to add weights to each buoy which will inform a random.random.choice function with weighted probabilities. This will allow the system to take photos of the most beautiful buoys more often.



The artist will use an ESRGAN on the photo sets that they think are most beautiful and enhance their quality. If they go well together then the artist will be able to create a video of the photo set or a panorama of the photo set.

- Step One - Evaluation of the photo sets
- Step Two - ESRGAN on the photo sets
- Step Three - Video creation
- Step Four - Panorama creation



In [13]:
import cv2
import os
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time
import random
import imutils
# import pillow
from PIL import Image

In [14]:
class Artist:
    def __init__(self):
        self.folder = 'images'
    
    def make_panorama(self, images):
        # take a list of images and stitch them together with cv2
        stitcher = cv2.createStitcher() if imutils.is_cv3() else cv2.Stitcher_create()
        (status, stitched) = stitcher.stitch(images)
        if status == 0:
            return stitched
        else:
            return None
    

In [15]:
def artist_eval(image_path):
    # get the image
    img = Image.open(image_path)
    # get the image size
    width, height = img.size
    # get the pixel values for the center of the image
    pixel_values = img.getpixel((int(width/2), int(height/2)))
    # get the pixel values for the top left corner of the image
    upper_left = img.getpixel((0, 0))
    # get the pixel values for the top right corner of the image
    upper_right = img.getpixel((width-1, 0))
    
    # get the pixel values for the centers of the six panels in the image
    panel_1 = img.getpixel((int(width/12), int(height/2)))
    panel_2 = img.getpixel((int(width/4), int(height/2)))
    panel_3 = img.getpixel((int(width/2), int(height/2)))
    panel_4 = img.getpixel((int(width*3/4), int(height/2)))
    panel_5 = img.getpixel((int(width*5/6), int(height/2)))
    panel_6 = img.getpixel((int(width*11/12), int(height/2)))

    # a panorama is best when the panels are similar in color
    # let's see how close the panels are to each other
    # we'll use the mean squared error
    mse_1 = np.mean((np.array(panel_1) - np.array(panel_2)) ** 2)
    mse_2 = np.mean((np.array(panel_2) - np.array(panel_3)) ** 2)
    mse_3 = np.mean((np.array(panel_3) - np.array(panel_4)) ** 2)
    mse_4 = np.mean((np.array(panel_4) - np.array(panel_5)) ** 2)
    mse_5 = np.mean((np.array(panel_5) - np.array(panel_6)) ** 2)

    # the mean squared error is a good measure of how similar the panels are
    # the lower the mse, the more similar the panels are
    # let's take the average of the mse's
    mse = (mse_1 + mse_2 + mse_3 + mse_4 + mse_5) / 5

    # we will make a panorama if the mse is less than 100
    # this is a pretty arbitrary number, but it seems to work well
    if mse < 100:
        return True
    else:
        return False
    

In [17]:
vincent = Artist() # create an instance of the Artist class
# for each folder in the images/panels folder, stitch the images together and save them to the images/panoramas folder with the same name as the folder + panorama.png
for folder in os.listdir('images/panels'):
    if folder == '.DS_Store':
        continue
    images = []
    for image in tqdm(os.listdir(f'images/panels/{folder}')):
        if image == '.DS_Store':
            continue
        if artist_eval(f'images/panels/{folder}/{image}'): # if the artist decides to make a panorama (True)
            images.append(cv2.imread(f'images/panels/{folder}/{image}'))
            try:
                panorama = vincent.make_panorama(images)
                cv2.imwrite(f'images/panoramas/{folder}_panorama.png', panorama)
            except:
                ##print(f'Could not create panorama for {folder}')
                pass
        else:
            #print(f'Could not create panorama for {folder}')
            pass


100%|██████████| 6/6 [00:00<00:00, 170.60it/s]
